<a href="https://colab.research.google.com/github/biaferre/bioinspirada/blob/main/Finding_Primes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encontrando números primos

> Disciplina de Tópicos Avançados em IA: Algoritmos Bioinspirados | CIn UFPE 2025.1

> Grupo: Beatriz Férre e Ronald Silva

Exercício em Python para encontrar primos dentro de um intervalo de números inteiros com mais de 6 casas decimais. Experimentações com testes de primalidade, tamanho populacional e intervalo da busca, métodos de seleção (torneio x roleta), representações com inteiros e funções de combinação e mutação pra inteiros.



### Parâmetros

In [96]:
import random

MIN_VAL = 10**8
MAX_VAL = 10**12
POP_SIZE = 5
GEN = 100

### Código base
Fornecida na monitoria para ser aprimorada nas seções subsequentes.

In [ ]:
# Checar se o número é primo
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

# Função de fitness: 1 se primo, senão penaliza pelo número de divisores
def fitness(n):
    if is_prime(n):
        return 1000  # grande valor de fitness
    else:
        divisores = sum(1 for i in range(2, n) if n % i == 0)
        return 1 / (divisores + 1)

# Inicializa a população de inteiros
def init_population(size, min_val, max_val):
    return [random.randint(min_val, max_val) for _ in range(size)]

# Seleção: Torneio binário
def selection(population, fitnesses):
    selected = []
    for _ in range(len(population)):
        if len(population) < 2:
            return population  # evita erro se houver apenas 1 indivíduo
        i, j = random.sample(range(len(population)), 2)
        selected.append(population[i] if fitnesses[i] > fitnesses[j] else population[j])
    return selected

# Crossover: combinação de bits
def crossover(parent1, parent2):
    p1 = format(parent1, 'b')
    p2 = format(parent2, 'b')
    max_len = max(len(p1), len(p2))
    p1 = p1.zfill(max_len)
    p2 = p2.zfill(max_len)
    point = random.randint(1, max_len - 1)
    child_bin = p1[:point] + p2[point:]
    return int(child_bin, 2)

# Mutação: troca aleatória de bits
def mutate(n, mutation_rate=0.05): # aumentar 0.2
    bits = list(format(n, 'b'))
    for i in range(len(bits)):
        if random.random() < mutation_rate:
            bits[i] = '1' if bits[i] == '0' else '0'
    return int(''.join(bits), 2)

# Algoritmo evolutivo
def evolutionary_prime_search(pop_size=10, min_val=10**5, max_val=10**6, generations=100):
    population = init_population(pop_size, min_val, max_val)

    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]

        # Melhor indivíduo da geração
        best_idx = fitnesses.index(max(fitnesses))
        best_ind = population[best_idx]
        best_fit = fitnesses[best_idx]

        print(f"Geração {gen}: Melhor fitness = {best_fit:.5f}, Melhor indivíduo = {best_ind}")

        # Verifica se encontrou um primo
        if best_fit == 1000:
            print(f"\n✅ Primo encontrado na geração {gen}: {best_ind}")
            return best_ind

        # Seleção
        selected = selection(population, fitnesses)

        # Crossover com reposição até manter o tamanho original
        next_population = []
        while len(next_population) < pop_size:
            p1, p2 = random.sample(selected, 2)
            child = crossover(p1, p2)
            next_population.append(child)

        # Mutação
        population = [mutate(ind) for ind in next_population]

    print("\n❌ Nenhum primo encontrado após todas as gerações.")
    return None

# Executa o algoritmo
evolutionary_prime_search()

### Experimentações

In [97]:
# Checar se o número é primo
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0:
            return False
    return True

# Função de fitness: 1 se primo, senão penaliza pelo número de divisores
def fitness(n):
    if is_prime(n):
        return 1000  # grande valor de fitness
    else:
        divisores = sum(1 for i in range(2, n) if n % i == 0)
        return 1 / (divisores + 1)

# Inicializa a população de inteiros
def init_population(size, min_val, max_val):
    population = []
    while len(population) < size:
      val = random.randint(min_val, max_val)
      if (val % 2 != 0): # adicionando apenas ímpares
        population.append(val)
    return population

# Seleção: Roleta
def selection(population, fitnesses):
    total_fitness = sum(fitnesses)
    individual_probabilities = []
    for i in population:
      i_probability = fitnesses[population.index(i)] / total_fitness
      individual_probabilities.append((i, i_probability))

    print(individual_probabilities)

    p1 = 0
    roulette_num = random.random()
    for i in individual_probabilities:
      roulette_num -= i[1]
      if roulette_num <= 0:
        p1 = i
        break

    p2 = 0
    roulette_num = random.random()
    for i in individual_probabilities:
      roulette_num -= i[1]
      if roulette_num <= 0:
        p2 = i
        break

    return p1, p2

# Crossover: recombinação intermediária
def crossover(p1, p2):
    d = random.uniform(0, 0.5)
    child = int(p1 * d + p2 * (1 - d))
    return child

# Mutação: troca aleatória de bits
def mutate(n, mutation_rate=0.05): # aumentar 0.2
    bits = list(format(n, 'b'))
    for i in range(len(bits)):
        if random.random() < mutation_rate:
            bits[i] = '1' if bits[i] == '0' else '0'
    return int(''.join(bits), 2)

In [98]:

# Algoritmo evolutivo
def evolutionary_prime_search(pop_size=POP_SIZE, min_val=MIN_VAL, max_val=MAX_VAL, generations=GEN):
    population = init_population(pop_size, min_val, max_val)

    for gen in range(generations):
        fitnesses = [fitness(ind) for ind in population]

        # Melhor indivíduo da geração
        best_idx = fitnesses.index(max(fitnesses))
        best_ind = population[best_idx]
        best_fit = fitnesses[best_idx]

        print(f"Geração {gen}: Melhor fitness = {best_fit:.5f}, Melhor indivíduo = {best_ind}")

        # Verifica se encontrou um primo
        if best_fit == 1000:
            print(f"\n✅ Primo encontrado na geração {gen}: {best_ind}")
            return best_ind

        # Seleção
        selected = selection(population, fitnesses)
        print(selected)
        p1 = selected[0][0]
        p2 = selected[1][0]

        # Crossover com reposição até manter o tamanho original
        next_population = []
        while len(next_population) < pop_size:
            child = crossover(p1, p2)
            next_population.append(child)

        # Mutação
        population = [mutate(ind) for ind in next_population]

    print("\n❌ Nenhum primo encontrado após todas as gerações.")
    return None


In [ ]:
# Executa o algoritmo
evolutionary_prime_search()